**I modified this to get 1000 doctors for SF; saved into 'yelp_1000_doc_sf.json'**

#### Yelp Fusion API      
1. https://github.com/Yelp/yelp-fusion/tree/master/fusion/python     
2. $pip install -r requirements.txt
3. By default, you’ll receive 25,000 calls per day. If you expect to hit this rate limit, please contact api@yelp.com with links to or screenshots of your app or website.      
https://www.yelp.com/developers/faq

### confidentials I created for our MedHub app
App ID: ezS3PAsVkMfPq5bqVvAXUg      
App Secret: zZIaiWTiyuUvuXM7EYMUvxt52jdKVftFl5TjIR4ByCt2rwzR9txyzV0EkkmHlJJt

In [44]:
# OAuth credential placeholders that must be filled in by users.
# You can find them on
# https://www.yelp.com/developers/v3/manage_app
CLIENT_ID = 'ezS3PAsVkMfPq5bqVvAXUg'
CLIENT_SECRET = 'zZIaiWTiyuUvuXM7EYMUvxt52jdKVftFl5TjIR4ByCt2rwzR9txyzV0EkkmHlJJt'

In [45]:
from __future__ import print_function

import json
import pprint
import requests
import sys
import urllib

from urllib2 import HTTPError
from urllib import quote
from urllib import urlencode

In [46]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'

### Yelp functions

In [47]:
def obtain_bearer_token(host, path):
    """Given a bearer token, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        str: OAuth bearer token, obtained using client_id and client_secret.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    bearer_token = response.json()['access_token']
    return bearer_token


def request(host, path, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }

    #print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(bearer_token, term, location, off):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """
    
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': 50,
        'offset': off
    }
    return request(API_HOST, SEARCH_PATH, bearer_token, url_params=url_params)


def get_business(bearer_token, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, bearer_token)


def query_api_json(term, location, outputfile, search_limit):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    bearer_token = obtain_bearer_token(API_HOST, TOKEN_PATH)
    
    for off in range(0, search_limit, 50):
        print(u'Result saved into json: No.{0} :'.format(off + 1))
        response = search(bearer_token, term, location, off)

        businesses = response.get('businesses')

        if not businesses:
            #print(u'No businesses for "{0}" in "{1}" found.'.format(term, location))
            return
        else:
            print(u'{0} businesses found, querying business info'.format(len(businesses)))
            f = open(outputfile, 'a')
            #count = 1
            for b in businesses:
                #if count % 10 == 1: print(u'Result saved into json: No.{0} :'.format(count))
                business_id = b['id']
                try:
                    response = get_business(bearer_token, business_id)
                    #print(u'Result for business {0} saved into json:'.format(business_id))
                    json.dump(response, f)
                    f.write("\n")
                    #count += 1
                except:
                    pass
            print(u'Done... {0} businesses have been added to the json file'.format(len(businesses)))
            f.close()

### Save jsons to the same file

In [48]:
# Defaults for our simple example.
term = 'doctor'
location = 'San Francisco, CA'
SEARCH_LIMIT = 1000

In [49]:
try:
    query_api_json(term, location, 'yelp_1000_doc_sf.json', SEARCH_LIMIT)
except HTTPError as error:
    sys.exit(
        'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
            error.code,
            error.url,
            error.read(),
        )
    )

Result saved into json: No.1 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.51 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.101 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.151 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.201 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.251 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.301 :
50 businesses found, querying business info
Done... 50 businesses have been added to the json file
Result saved into json: No.351 :
50 businesses found, querying business info
Do